IMPORTING THE REQUIRED LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

READING THE XML FILES PATH FROM THE CSV FILE 

In [2]:
df = pd.read_csv('Labels.csv')
df

,filepath,xmin,xmax,ymin,ymax
0,./xml\Cars0.xml,228,425,130,171
1,./xml\Cars1.xml,126,266,125,154
2,./xml\Cars10.xml,106,304,16,145
3,./xml\Cars102.xml,31,324,138,202
4,./xml\Cars104.xml,195,243,267,282
...,...,...,...,...,...
107,./xml\Cars94.xml,172,261,206,252
108,./xml\Cars96.xml,130,264,123,155
109,./xml\Cars97.xml,92,303,100,199
110,./xml\Cars98.xml,78,252,189,281


PARSING THE XML FILES TO GET THE IMAGE NAME

In [3]:
# The image name is stored as the <filename> 
import xml.etree.ElementTree as xet
#Let's create a function to parse the image name and the generate the image path
def getImagePath(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text # Parse the imagename from the xml
    filepath_image = os.path.join('./img',filename_image) #Join the image name with the path to generate complete path for the image
    return filepath_image
#Let's appply the getImagePath function to the entire lsit of xml files to generate the list of image paths
image_path = list(df['filepath'].apply(getImagePath))

VERIFYING THAT IMAGE READING CAN BE DONE

In [4]:
filepath = image_path[0]
#Using the OpenCV module to read image in BGR format
img = cv2.imread(filepath)
# Let's see the boxed window for the img region to be read
cv2.rectangle(img,(228,130),(425,171), (0,255,0), 3)
# Create a window named "example" and set it's size as WINDOW_NORMAL
cv2.namedWindow('example',cv2.WINDOW_NORMAL) 
cv2.imshow('example',img)
cv2.waitKey(0) #Close the window on click of any key
cv2.destroyAllWindows() #Close all the windows

DATA PREPROCESSING

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
data = [] #Creating list for storing the array of images as elements
output = [] #Creating list for storing the bouding boxes 
labels = df.iloc[:,1:].values #Collect the dimension of the red box for OCR in DataFrame labels
for ind in range (len(image_path)): #For all the images
    image = image_path[ind] # Transfer each image path to the image variable
    img_arr = cv2.imread(image) # read the image in BGR format using OpenCV 
    h,w,d = img_arr.shape # Dimension of the image in numbers
    # Performing the Preprocessing
    load_image = load_img(image, target_size=(224,224)) # Load the image from the 'image' file path and resize it to a target size of 224x224 pixels
    load_image_arr = img_to_array(load_image) # Convert the loaded image (in PIL format) to a NumPy array
    # Performing Normalization on hte load_image_arr
    norm_load_image_arr = load_image_arr/255.0
    # Performing the Normalization on the boundbox coordinates
    xmin,xmax,ymin,ymax = labels[ind] #Tuple Unpacking done in numpy array labels Can't be done in DataFrame
    nxmin,nxmax = xmin/w, xmax/w #normalizing with respect to the width
    nymin,nymax = ymin/h, ymax/h #normalizing with respect to the height
    label_norm = (nxmin, nxmax, nymin, nymax) #Tuple creation for the bouding box with normalized dimensions
    data.append(norm_load_image_arr)
    output.append(label_norm)

CONVERTING THE data AND output LIST INTO NUMPY ARRAY AND ASSIGNING THEM TO X AND y
<br>TRAIN TEST SPLIT

In [6]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size= 0.2, random_state=0)

DEEP LEARNING MODEL

In [7]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [8]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
inception_resnet.trainable = False
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)
model = Model(inputs=inception_resnet.input, outputs=headmodel)

COMPILING THE MODEL

In [11]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-40))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 111, 111, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 111, 111, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 111, 111, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 109, 109, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 109, 109, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 109, 109, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 109, 109, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 109, 109, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 109, 109, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 54, 54, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 54, 54, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 54, 54, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 54, 54, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 52, 52, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

TRAINING THE RESNETV2 MODEL

In [12]:
from tensorflow.keras.callbacks import TensorBoard

In [13]:
tfb = TensorBoard('object_detection')

In [14]:
history = model.fit(x=X_train, y=y_train, batch_size=10, epochs=100, validation_data=(X_test,y_test), callbacks=[tfb])

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - loss: 0.1149 - val_loss: 0.1444
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1068 - val_loss: 0.1444
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1062 - val_loss: 0.1444
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1073 - val_loss: 0.1444
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1055 - val_loss: 0.1444
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1188 - val_loss: 0.1444
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1086 - val_loss: 0.1444
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1049 - val_loss: 0.1444
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1202 - val_loss: 0.1444
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1156 - val_loss: 0.1444
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1143 - val_loss: 0.1444
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1057 - val_loss: 0.1444
E

In [17]:
history = model.fit(x=X_train, y=y_train, batch_size=10, epochs=200, validation_data=(X_test,y_test), callbacks=[tfb], initial_epoch=100)

Epoch 101/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 0.1128 - val_loss: 0.1444
Epoch 102/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1156 - val_loss: 0.1444
Epoch 103/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1073 - val_loss: 0.1444
Epoch 104/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1142 - val_loss: 0.1444
Epoch 105/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1111 - val_loss: 0.1444
Epoch 106/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1109 - val_loss: 0.1444
Epoch 107/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1088 - val_loss: 0.1444
Epoch 108/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1145 - val_loss: 0.1444
Epoch 109/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1166 - val_loss: 0.1444
Epoch 110/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1121 - val_loss: 0.1444
Epoch 111/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1078 - val_loss: 0.1444
Epoch 112/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 0.1159

In [19]:
model.save('./models/object_detection.keras')